In [9]:
import pandas as pd
import datetime

In [2]:
ratings = pd.read_csv("./enrich/ratings.csv")
ratings

,userID,itemID,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
movies = pd.read_csv("./enrich/movies.csv")
movies

,itemID,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [4]:
list_of_genres = ['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [5]:
for genre in list_of_genres:
    movies[genre] = movies['genre'].apply(lambda x: 1 if genre in x.split('|') else 0)
movies = movies.drop('genre', axis=1)

In [6]:
movies.to_csv("./enrich/movies.csv")

In [7]:
curatedData = ratings.merge(movies, on="itemID", how="inner")

In [8]:
curatedData.to_csv("./curated/ratings_info.csv", index=False)

In [12]:
u_col = ["userID", "Gender", "Age", "Occupation", "ZipCode"]
users = pd.read_csv("./data/movielens_1M/users.dat", sep="::",
                    names=u_col)
users

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_34640\2899099271.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv("./data/movielens_1M/users.dat", sep="::",


,userID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [16]:
age_dict = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+"
}

# Tạo từ điển cho dữ liệu về nghề nghiệp
occupation_dict = {
    0: "other or not specified",
    1: "academic/educator",
    2: "artist",
    3: "clerical/admin",
    4: "college/grad student",
    5: "customer service",
    6: "doctor/health care",
    7: "executive/managerial",
    8: "farmer",
    9: "homemaker",
    10: "K-12 student",
    11: "lawyer",
    12: "programmer",
    13: "retired",
    14: "sales/marketing",
    15: "scientist",
    16: "self-employed",
    17: "technician/engineer",
    18: "tradesman/craftsman",
    19: "unemployed",
    20: "writer"
}

# Chuyển từ điển thành DataFrame
age_df = pd.DataFrame(list(age_dict.items()), columns=['Age', 'age_group'])
occupation_df = pd.DataFrame(list(occupation_dict.items()), columns=['Occupation', 'occupation_name'])

In [17]:
age_df

,Age,age_group
0,1,Under 18
1,18,18-24
2,25,25-34
3,35,35-44
4,45,45-49
5,50,50-55
6,56,56+


In [19]:
new_users = users.merge(age_df, on="Age")
new_users

,userID,Gender,Age,Occupation,ZipCode,age_group
0,1,F,1,10,48067,Under 18
1,19,M,1,10,48073,Under 18
2,51,F,1,10,10562,Under 18
3,75,F,1,10,01748,Under 18
4,86,F,1,10,54467,Under 18
...,...,...,...,...,...,...
6035,5998,M,18,4,61820,18-24
6036,6008,M,18,4,78705,18-24
6037,6027,M,18,4,20742,18-24
6038,6028,M,18,4,94133,18-24


In [20]:
new_users = new_users.merge(occupation_df, on="Occupation")


In [22]:
new_users["Age"] = new_users["age_group"]
new_users["Occupation"] = new_users["occupation_name"]

In [24]:
users = new_users.drop(columns=["age_group", "occupation_name"])

In [25]:
users.to_csv("./enrich/users.csv")

,userID,Gender,Age,Occupation,ZipCode
0,1,F,Under 18,K-12 student,48067
1,19,M,Under 18,K-12 student,48073
2,51,F,Under 18,K-12 student,10562
3,75,F,Under 18,K-12 student,01748
4,86,F,Under 18,K-12 student,54467
...,...,...,...,...,...
6035,4338,F,18-24,customer service,46222
6036,5320,F,18-24,customer service,43201
6037,5348,M,18-24,customer service,55414
6038,5350,M,18-24,customer service,55344
